In [1]:
import requests

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
from elasticsearch import Elasticsearch

In [3]:
es_client = Elasticsearch('http://localhost:9200')

In [36]:
# **Question1**

In [5]:
es_client.info()

ObjectApiResponse({'name': '802504b82092', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'c2cWCbSnTyiG3Ad_xJuwnw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [37]:
# **Question2**

In [7]:
# index_settings = {
#     "settings": {
#         "number_of_shards": 1,
#         "number_of_replicas": 0
#     },
#     "mappings": {
#         "properties": {
#             "text": {"type": "text"},
#             "section": {"type": "text"},
#             "question": {"type": "text"},
#             "course": {"type": "keyword"}
#         }
#     }
# }
#
# index_name = "course-questions"
#
# es_client.indices.create(index=index_name, body=index_settings)


In [8]:
index_name = "course-questions"

# Check if the index exists and delete it if it does
if es_client.indices.exists(index=index_name):
    print(f"Deleting existing index: {index_name}")
    es_client.indices.delete(index=index_name)
    print(f"Index {index_name} deleted.")

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}

print(f"Creating index: {index_name}")
es_client.indices.create(index=index_name, body=index_settings)
print(f"Index {index_name} created successfully.")

Creating index: course-questions
Index course-questions created successfully.


In [9]:
from tqdm.auto import tqdm

In [10]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [38]:
# **Question3**

In [12]:
query_q3 = 'How do execute a command on a Kubernetes pod?'

In [14]:
search_query_q3 = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query_q3,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
        }
    }
}

In [15]:
response_q3 = es_client.search(index=index_name, body=search_query_q3)

In [16]:
results_docs_q3_source = []

for hit in response_q3['hits']['hits']:
    results_docs_q3_source.append(hit['_source'])

results_docs_q3_source

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke',
  'section': '10. Kubernetes and TensorFlow Serving',
  'question': 'Kubernetes-dashboard',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t

In [17]:
results_docs_q3_score = []

for hit in response_q3['hits']['hits']:
    results_docs_q3_score.append(hit['_score'])

results_docs_q3_score

[44.50556, 35.433445, 33.70974, 33.2635, 32.589073]

In [39]:
# **Question4**

In [19]:
query_q4 = 'How do copy a file to a Docker container?'

In [20]:
search_query_q4 = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query_q4,
                    "fields": ["question^4", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [21]:
response_q4 = es_client.search(index=index_name, body=search_query_q4)

In [22]:
results_docs_q4_source = []

for hit in response_q4['hits']['hits']:
    results_docs_q4_source.append(hit['_source'])

#results_docs_q4_source[2]
print(results_docs_q4_source[2]['question'])

How do I copy files from a different folder into docker container’s working directory?


In [42]:
# **Question5**

In [26]:
from openai import OpenAI

# Define the path to your API key file
# It's good practice to keep this file outside your project's version control (e.g., in .gitignore)
api_key_file_path = "openai_api_key.txt" # Or a path like 'config/api_key.txt'

api_key = None # Initialize api_key

try:
    with open(api_key_file_path, 'r') as f:
        api_key = f.read().strip() # Read the key and remove any leading/trailing whitespace
except FileNotFoundError:
    print(f"Warning: API key file not found at {api_key_file_path}. Checking environment variable instead.")
    # Fallback to environment variable if the file isn't found
    api_key = os.environ.get("OPENAI_API_KEY")
except Exception as e:
    raise IOError(f"Error reading API key from file {api_key_file_path}: {e}")

# Check if the API key was found either from file or environment variable
if api_key is None:
    raise ValueError("OPENAI_API_KEY not found. Please set it in 'openai_api_key.txt' or as an environment variable.")

# Initialize the OpenAI client with the fetched key
client = OpenAI(api_key=api_key)

print("OpenAI client initialized.")

OpenAI client initialized.


In [27]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    print(f"The length of the prompt for question5 is {len(prompt)}")
    return prompt

In [28]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [29]:
    prompt = build_prompt (query_q4, results_docs_q4_source)
    llm(prompt)

The length of the prompt for question5 is 2424


'To copy a file to a Docker container, you can use the `docker cp` command. The basic syntax is as follows:\n\n```bash\ndocker cp /path/to/local/file_or_directory <container_id>:/path/in/container\n```\n\nThis command allows you to copy files from your local machine into a running Docker container.'

In [41]:
# **Question6**

In [31]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

# Encode the prompt to get the list of token IDs
tokens = encoding.encode(prompt)

# The number of tokens is the length of this list
num_tokens = len(tokens)

print(f"The prompt has {num_tokens} tokens.")

The prompt has 538 tokens.
